In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar -xf spark-3.5.0-bin-hadoop3.tgz
!rm spark-3.5.0-bin-hadoop3.tgz

!pip install -q pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from itertools import combinations
import os
from pyspark.ml.fpm import FPGrowthModel
from pyspark.sql.functions import array, lit, col, explode, desc

In [ ]:
class RecommendModel:
    def __init__(self):
        self.cd_pill_intake = []
        self.df_pill = None
        self.spark = SparkSession \
                    .builder \
                    .appName("spark_pill") \
                    .config("spark.jars", "/content/drive/MyDrive/model/mysql-connector-j-8.1.0.jar") \
                    .getOrCreate()

        models = {}
        for dir_name in os.listdir('/content/drive/My Drive/model'):
            if ".model" in dir_name:
                gender, mem_age = dir_name.split("_")[:2]
                path = os.path.join('/content/drive/My Drive/model', dir_name)
                model = FPGrowthModel.load(path)
                models[(gender, mem_age)] = model
        self.models = models

    # ====================================================================== #
    ### mariadb 데이터 조회 함수
    def mysql_select(self, spark:SparkSession, query:str):
        # [mariadb 설정 변수]
        db_domain = "54.180.91.68"
        db_port = "3306"
        database = "dw"
        user = "batch"
        password = "Data1q2w3e4r!!"

        return spark \
            .read \
            .format("jdbc") \
            .option("driver", "com.mysql.jdbc.Driver") \
            .option("url", f"jdbc:mysql://{db_domain}:{db_port}/{database}?serverTimezone=UTC") \
            .option("query", query) \
            .option("user", user) \
            .option("password", password) \
            .load()
    # ====================================================================== #
    ######### [DB 사용자 정보 조회] ###########
    def getMemData(self, mem):

        # [데이터 설정 변수]
        # - 로그인 회원 설정
        mem = mem
        # - 한 달 전 기준 설정
        before_date = (datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d')
        #-------------------------------------------------------#
        # [사용자 정보 조회]
        sql = f"""
            SELECT mem_gen, mem_age
            FROM mem_detail
            WHERE mem_email = "{mem}"
        """
        ### 데이터 조회
        mem_df = self.mysql_select(self.spark, sql)
        # - 회원 성별 값 추출
        mem_gen = mem_df.take(1)[-1].mem_gen
        # - 회원 나이 값 추출
        mem_age = mem_df.take(1)[-1].mem_age
        #-------------------------------------------------------#
        # [섭취 영양제 정보 조회]
        # - 빠른 실행을 위해 회원 필터링하는 서브쿼리 정의 후 pill 정보 join
        sql = f"""
            WITH filtered_prtn AS (
              SELECT prtn_mem, prtn_fin.prtn_id, prtn_nm, prtn_cat, prtn_tag, prtn_set, prtn_reps,
                     prtn_sdate, prtn_edate, prtn_time, prtn_alram, prtn_day, fin_prtn_time
              FROM prtn_setting
              JOIN prtn_fin
                  ON prtn_setting.prtn_id = prtn_fin.prtn_id
              WHERE prtn_mem='{mem}'
                  AND DATE_FORMAT(fin_prtn_time, '%Y-%m-%d') > "{before_date}"
            )

            SELECT DISTINCT cat_nm, cmb_nutr, pill_nm
            FROM pill_cat
            JOIN pill_cmb
                ON pill_cat.cat_cd = pill_cmb.cmb_cat
            JOIN pill_prod
                ON pill_cmb.cmb_pill = pill_prod.pill_cd
            JOIN filtered_prtn
                ON filtered_prtn.prtn_nm = pill_prod.pill_cd
        """

        ### 데이터 조회
        pill_df = self.mysql_select(self.spark, sql)
        self.df_pill = pill_df
        # - 섭취 영양제 대분류 리스트로 추출
        pill_intake = pill_df.select("cat_nm").rdd.flatMap(lambda x: x).distinct().collect()
        # - 섭취 영양제 cd 리스트로 추출
        cd_pill_intake = pill_df.select("cmb_nutr").rdd.flatMap(lambda x: x).distinct().collect()
        self.cd_pill_intake = cd_pill_intake

        return mem_gen, mem_age, pill_intake
    # ====================================================================== #
    ######### [연관 규칙 예측 수행 함수] ###########
    def getModel(self, mem_gen, mem_age, pill_intake):
        gender_map = {'male': '남자', 'female': '여자'}
        gender = gender_map.get(mem_gen)
        age_model = f"{mem_age}.model"
        model = self.models.get((gender, age_model))
        if not model:
            return f"모델을 찾을 수 없습니다. ({gender}, {mem_age})"

        # 불러온 모델 기반 규칙 찾기
        rules = model.associationRules

        items_array = array(*[lit(pill) for pill in pill_intake]) # 입력한 pill 리스트 -> ArrayType 변환
        relevant_rules = rules.filter(col("antecedent") == items_array) # 규칙 필터링

        # 규칙이 없는 경우 처리
        if relevant_rules.count() == 0:
            return "해당 아이템에 대한 규칙을 찾을 수 없습니다."

        # 향상도로 1 초과하는 값 필터링하고 정렬
        top_lift = relevant_rules.filter(col("lift") > 1).orderBy(desc("lift")).head(1)
        # 향상도가 1을 초과하지 않는 경우 처리
        if not top_lift:
            return "향상도가 1 초과인 규칙을 찾을 수 없습니다."

        top_consequent = top_lift[0]['consequent']
        # top_consequent는 배열형태임 => 첫 번째 반환
        if top_consequent:
            recommend_cat = top_consequent[0]
            return top_consequent[0]  # 추천 아이템의 이름을 문자열로 반환
        else:
            return "추천할 아이템이 없습니다."
    # ====================================================================== #
    ######### [DB 영양제 정보 조회] ###########
    def getPillData(self, recommend_cat):

        #### 최종 결과 변수 : result
        result = []

        # [부작용 우려 조합 조회]
        sql = f"""
            SELECT sideeff_cd1, sideeff_cd2
            FROM pill_sideeff
        """

        ### 데이터 조회
        dont_intake_df = self.mysql_select(self.spark, sql)
        # - 부작용 우려 조합 튜플로 추출
        dont_intake_comb = dont_intake_df.rdd.map(lambda row: \
                                                  (row["sideeff_cd1"],row["sideeff_cd2"])).collect()
        # - 사용자가 섭취한 성분으로 가능한 2개의 조합 추출
        combs = tuple(combinations(self.cd_pill_intake,2))
        #-------------------------------------------------------#
        # - 부작용 성분이 있는 제품명 조회
        prod1_2 = []
        for comb in combs:
            if comb in dont_intake_comb:
                # 부작용 제품 1
                prod1 = self.df_pill.filter(self.df_pill["cmb_nutr"]==comb[0]) \
                                     .take(1)[-1].pill_nm
                # 부작용 제품 2
                prod2 = self.df_pill.filter(self.df_pill["cmb_nutr"]==comb[1]) \
                                     .take(1)[-1].pill_nm
                if prod1!=prod2 : prod1_2.append((prod1, prod2))
            else : continue

        ### 부작용 제품 결과 리포팅 양식으로 추출
        dont_result = [f"❗ : \"{prod1}\" & \"{prod2}\" \n    함께 섭취 시 부작용이 있을 수 있어요!"\
                       for prod1, prod2 in set(prod1_2)]
        result += dont_result
        #-------------------------------------------------------#
        # [추천 제품 조회]
        recommend_cat = recommend_cat
        # [추천 제품 조회]
        sql = f"""
            WITH filtered_nutn AS (
              SELECT cmb_pill, cat_nm
              FROM pill_cat
              JOIN pill_cmb
                  ON pill_cat.cat_cd = pill_cmb.cmb_cat
              WHERE cat_nm='{recommend_cat}'
            )

            SELECT DISTINCT cat_nm, pill_nm
            FROM filtered_nutn
            JOIN pill_prod
                ON filtered_nutn.cmb_pill = pill_prod.pill_cd
            WHERE pill_nm NOT LIKE '%(단종)%'
            ORDER BY pill_rvnum DESC, pill_rv DESC LIMIT 3
        """
        ### 데이터 조회
        recommend_df = self.mysql_select(self.spark, sql)
        recommend_result = recommend_df.rdd.map(lambda row: \
                           f"👍 추천제품 : \"{row['cat_nm']}\"군의 \"{row['pill_nm']}\"").collect()
        result += recommend_result

        return result


In [ ]:
recom = RecommendModel()

In [ ]:
mem_gen, mem_age, pill_intake = recom.getMemData("moonsee12@naver.com")
recommend_result = recom.getModel(mem_gen, mem_age, pill_intake)
result = recom.getPillData(recommend_result)
result

['ab', 'ae', 'af', 'ai', 'ak', 'al', 'an', 'aq', 'as', 'av', 'bd', 'bm', 'ca', 'di', 'do', 'aa', 'ac', 'ad', 'ah', 'aj', 'am', 'ao', 'ap', 'at', 'au', 'aw', 'az', 'bc', 'cx', 'cy', 'de'] (('ab', 'ae'), ('ab', 'af'), ('ab', 'ai'), ('ab', 'ak'), ('ab', 'al'), ('ab', 'an'), ('ab', 'aq'), ('ab', 'as'), ('ab', 'av'), ('ab', 'bd'), ('ab', 'bm'), ('ab', 'ca'), ('ab', 'di'), ('ab', 'do'), ('ab', 'aa'), ('ab', 'ac'), ('ab', 'ad'), ('ab', 'ah'), ('ab', 'aj'), ('ab', 'am'), ('ab', 'ao'), ('ab', 'ap'), ('ab', 'at'), ('ab', 'au'), ('ab', 'aw'), ('ab', 'az'), ('ab', 'bc'), ('ab', 'cx'), ('ab', 'cy'), ('ab', 'de'), ('ae', 'af'), ('ae', 'ai'), ('ae', 'ak'), ('ae', 'al'), ('ae', 'an'), ('ae', 'aq'), ('ae', 'as'), ('ae', 'av'), ('ae', 'bd'), ('ae', 'bm'), ('ae', 'ca'), ('ae', 'di'), ('ae', 'do'), ('ae', 'aa'), ('ae', 'ac'), ('ae', 'ad'), ('ae', 'ah'), ('ae', 'aj'), ('ae', 'am'), ('ae', 'ao'), ('ae', 'ap'), ('ae', 'at'), ('ae', 'au'), ('ae', 'aw'), ('ae', 'az'), ('ae', 'bc'), ('ae', 'cx'), ('ae', 'cy'), 

['❗ : "에너지 익스트림" & "하이키드 쵸코" \n    함께 섭취 시 부작용이 있을 수 있어요!']